# LSE Price and Volume Selenium Scraper
This programme scrapes prices and volumes from the London Stock Exchange Website by iterating through
the technical analysis page of various stocks and grabbing data from the charts.

In [1]:
import pandas as pd
from seleniumwire import webdriver
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import scipy.interpolate as si
import time
import requests
from bs4 import BeautifulSoup
from functools import partial
import numpy as np
import re
import random
chrome_path = "C:/Users/Graziano/Desktop/Python Files/Webdrivers/chromedriver.exe"
fire_path = "C:/Users/Graziano/Desktop/Python Files/Webdrivers/geckodriver.exe"

In [3]:
now = datetime.now()
today = f'{now.year}-{now.month}-{now.day}'
year, month, day = 5,0,0
start = f'{now.year - year}-{now.month - month}-{now.day - day}'
end_period = int(time.mktime(time.strptime(today, '%Y-%m-%d')))
start_period = int(time.mktime(time.strptime(start, '%Y-%m-%d')))

In [4]:
# base_url is the url that, combined with the four way key, gives the right url.
base_url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/exchange-insight/technical-analysis.html?'

In [5]:
# I do not set the options to run headlessly because it seems that I have to actually see the browsers
# render the charts in order to capture the data.
coptions = webdriver.ChromeOptions()
coptions.add_argument("--start-maximized")
foptions = webdriver.FirefoxOptions()

In [6]:
# tickerfo is the dataframe with tickers and four way keys in the fundamentals tab link
tickerfo = pd.read_csv(r'C:\Users\Graziano\Desktop\Trading\IndexScraper\ftse\ftse350_info.csv')

In [7]:
# I extract the tickers and four way keys for those keys that aren't nans
tickfo = tickerfo[~tickerfo.fund_link.isna()]
keys = tickfo.fund_link.apply(lambda x: str(x).split('?')[-1])
tickers = tickfo.tick

In [8]:
# using the keys I generate a list of appropriate urls
urls = [base_url + key for key in keys]

In [9]:
# I set the driver's scope and the number of tries and waits for the various functions.
scopes = ['.*(G|g)et(P|p)rices(W|w)ith(V|v)olume']
driver_tries, driver_wait = 20, 10
url_tries, url_wait = 5, 10
click_tries, click_wait = 5, 10

In [10]:
# I create an array of ones and zeros indicating whether the chrome or firefox webdriver will be in
# operation. I then turn that array into an array of ones, zeros, and minus ones, with the ones and minus
# ones indicating that a new driver will be initiated after closing the old one, and zeros indicating
# that the already existing driver will continue. This array will be zipped with the urls to control the
# execution of the programme
current = 0
stringFlow = ''
flow = []
for itm in range(len(urls)-1):
    if itm == 0:
        flow.append(0)
        stringFlow = stringFlow + '0'
    else:
        if np.random.uniform() < len(stringFlow.split('1')[-1])*0.04:
            if current >= 0:
                current = -1
                flow.append(current)
                stringFlow = stringFlow + str(current)
            else:
                current = 1
                flow.append(current)
                stringFlow = stringFlow + str(current)
        else:
            flow.append(0)
            stringFlow = stringFlow + '0'

In [11]:
def random_changer(random):
    '''This just changes -1 to 1 and vice versa'''
    if random == 1:
        return -1
    else:
        return 1

In [12]:
def random_webdriver_start(comptrol, scopes=scopes, chrome_path=chrome_path, 
                           fire_path=fire_path, chrome_options=coptions, fire_options=foptions):
    '''
    This function randomly returns a new chrome or firefox seleniumwire webdriver
    '''

    if comptrol == 1:
        try:
            driver = webdriver.Chrome(executable_path=chrome_path,
                                 options=coptions)
        except:
            #fp = webdriver.FirefoxProfile(r'C:\Users\Graziano\AppData\Roaming\Mozilla\Firefox\Profiles\t3cpv2qe.selenium')
            driver = webdriver.Firefox(executable_path=fire_path, 
                                       options=foptions)
    else:
        try:
            #fp = webdriver.FirefoxProfile(r'C:\Users\Graziano\AppData\Roaming\Mozilla\Firefox\Profiles\t3cpv2qe.selenium')
            driver = webdriver.Firefox(executable_path=fire_path, 
                                       options=foptions)
        except:
            driver = webdriver.Chrome(executable_path=chrome_path,
                                 options=coptions)

    driver.scopes = scopes
    return driver

In [13]:
def start_driver(comptrol=1, tries=driver_tries, wait=driver_wait):
    '''
    This function uses the start_driver function to try to open a webdriver driver_tries times and waits
    driver_wait seconds between tries.
    '''
    gate = 0
    while gate < tries:
        try:
            driver = random_webdriver_start(comptrol)
            gate = tries
            return driver
        except:
            print(f'had to try again with the browser. Comptrol:: {comptrol}')
            gate += 1
            if gate == tries:
                print('Could not start the driver','\n'+'---------------------------------------------')
                exit()
            else:
                time.sleep(driver_wait)

In [14]:
def try_url(driver,url,comptrol,url_tries=url_tries,url_wait=url_wait):
    '''
    try_url tries to get a url. If it fails, it tries again with a different browser. It tries url_tries
    times and waits for url_wait seconds between tries. It also globally modifies the boolean 'skip' if
    it fails to get the url.
    '''
    gate = 0
    global skip
    while gate < url_tries:
        try:
            driver.get(url)
            gate = url_tries
            skip = False
        except:
            gate += 1
            # If I reach the last try and the try block has failed to execute, then I print an error
            # message and set the skip parameter to True.
            if gate == url_tries:
                print(f'Could not get {url}')
                skip = True
            # for any other tries, I start a new browser by changing comptrol with random_changer and
            # employing start_driver.
            else:
                driver.quit()
                time.sleep(url_wait)
                comptrol = random_changer(comptrol)
                driver = start_driver(comptrol)

In [16]:
def clickety_click(driver,url,pre,click_tries=click_tries,click_wait=click_wait):
    '''
    This function moves to the 'add to portfolio' icon on the page and then mvoes left to get to the five
    year button and clicks. If there aren't two more requests relative to pre, then it tries again, and
    it tries again anyway if there is an error. It tries up to click_tries times and waits click_wait 
    seconds.
    '''
    # I use a while loop to make sure that the one year request has come through. This should hopefully
    # mean that the five year button has loaded. Making sure this request has come through should also
    # ensure that the rest of the code, which assumes two requests will come through, should work.
    gate = 0
    while len(driver.requests) - pre < 1 and gate < click_tries:
        time.sleep(click_wait)
        gate += 1
    # Once I have waited, I assign the new number of requests to pre_click, so I can detect if the
    # clicking has worked.
    pre_click = len(driver.requests)
    
    gate = 0
    while gate < click_tries:
        try:
            # portImg is a WebDriverWait that locates the portfolio icon using its xpath
            portImg = WebDriverWait(driver, 30).until(ec.visibility_of_element_located((
            By.XPATH, '//img[@alt="Add to Portfolio"]')))
            ActionChains(driver).move_to_element(portImg).move_by_offset(-100,0).click().perform()
            # I need to sleep in order to let the full request load
            time.sleep(5)
            # if the number of new requests is 1, then that means that the clicking has been successful.
            if len(driver.requests) - pre_click > 0:
                gate = click_tries
            else:
                gate += 1
        except:
            gate += 1
            # If I am on the last try then I print an error message.
            if gate == click_tries:
                print(f'could not clickety click {url}')
            else:
                time.sleep(click_wait)

In [17]:
def get_data(driver, pre, stonk, max_tries = 20, wait = 0.2):
    '''
    This function returns a list containng either nothing, or a pd dataframe with data from one of the
    lse graphs.
    '''
    tries = 0
    skip = False
    # I use a while loop to check max_tries times for the appearence of two extra requests, though this
    # condition is probably already met because I wait for two extra requests in clickety_click.
    while len(driver.requests) - pre < 2 and tries < max_tries:
        time.sleep(wait)
        tries += 1
    # I set the number of new requests as the 'new' variable in order to determine whether and which
    # data to get.
    new = len(driver.requests) - pre
    
    # If there are no new requests, then I return an empty list that concatenates with lse to get lse.
    # This is because I have no new requests, so no new data.
    if new == 0:
        return []
    # Otherwise, I wait until the last response is not None or until I have waited max_tries times.
    else:
        while driver.last_request.response is None and tries < max_tries:
            time.sleep(wait)
            tries += 1
        # If the last response is indeed not None, then I capture the json data from the response and
        # put it into a dataframe that I return in a list so as to be able to concatenate it. Note that
        # this could be just the 1 year request as well as the five year request.
        if driver.last_request.response is not None:
            jsn = driver.last_request.response.body
            data = json.loads(jsn)
            df = pd.DataFrame(data['d'], columns = ['date','close','open','high','low','close_again',
                                                    'volume'])
            return [pd.concat([df.set_index('date')], keys=[stonk], axis=1)]
        # Otherwise, if there is more than one request, then I check to see if the next to last request
        # (which will either be a five year or one year request) has a response and capture that instead.
        else:
            if new > 1 and driver.requests[-2].response is not None:
                jsn = driver.requests[-2].response.body
                data = json.loads(jsn)
                df = pd.DataFrame(data['d'], columns = ['date','close','open','high','low','close_again',
                                                        'volume'])
                return [pd.concat([df.set_index('date')], keys=[stonk], axis=1)]
            # However, if the new number of requests is one, then we already saw that the last request
            # had a None response, so there is no point in trying to capture data and I return []
            else:
                return []

In [18]:
# ------------------------------------------- MAIN PROGRAMME ---------------------------------------- #
# The main programme starts by going to the first url and gets its data and then deals with the rest
# of the urls through a for loop. It starts by creating an empty dataframe called 'lse' and setting
# comptrol to 1.
lse = pd.DataFrame()
comptrol = 1
# I start with a Chrome webdriver using comptrol.
driver = start_driver(comptrol)
# skip is assigned to False, and will be changed in try_url if I fail to get the url
skip = False
# I establish 'pre' which tells me how many requests have been made before any url getting or clicking
# takes place.
pre = len(driver.requests)
# I get the first url.
try_url(driver,urls[0],comptrol)

# if I couldn't get the url, then skip will have become true, so I skip the rest.
if skip:
    pass
# Otherwise, I proceed with the programme
else:
    # I check that I have not hit a 'page not found' page by checking the h1 tag, and skip the rest if
    # I have hit the page.
    if driver.find_element_by_tag_name('h1').text.lower() == 'page not found':
        pass
    # Otherwise, everything is fine and I clickety_click and then get the data
    else:
        clickety_click(driver,urls[0],pre)

        lse = pd.concat([lse] + get_data(driver,pre,tickers[0]), axis = 1)

# I begin a for loop based on a zip of previously defined lists.
for url, comptrol, stonk in list(zip(urls[1:], flow, tickers[1:]))[:5]:
    # if comptrol is zero, then that means that I am continuing with the existing driver
    if comptrol == 0:
        # I set skip to False just in case it was changed to True when getting the previous url
        skip = False
        # I define pre, which in this case is especially important because the number of requests will not
        # be zero
        pre = len(driver.requests)
        # The rest of the programme proceeds just as in the first instance.
        try_url(driver,url,comptrol)
        if skip:
            pass
        else:
            if driver.find_element_by_tag_name('h1').text.lower() == 'page not found':
                pass
            else:                
                clickety_click(driver,url,pre)
                lse = pd.concat([lse] + get_data(driver,pre,stonk), axis = 1)
    
    # This else block deals with the case when comptrol is not zero. I.E. whenI need to start with
    # another webdriver.
    else:
        # In this case I just quit the driver and start another one...
        driver.quit()
        driver = start_driver(comptrol)
        # ...then the rest of the programme is the same as before.
        skip = False
        pre = len(driver.requests)
        try_url(driver,url,comptrol)
        if skip:
            pass
        else:
            if driver.find_element_by_tag_name('h1').text.lower() == 'page not found':
                pass
            else:                
                clickety_click(driver,url,pre)
                lse = pd.concat([lse] + get_data(driver,pre,stonk), axis = 1)
# Finally I quit the driver.
driver.quit()

# ---------------------